In [3]:
import matplotlib.pyplot as plt
import benchmark_visualization as bv
import numpy as np
import glob
import os
import json

HOME_DIR = os.environ['HOME']

SHORT_TIMELIMIT = 600
LONG_TIMELIMIT = 3600

runs = {}
class BenchmarkRun:
    HEADER = "algorithm,graph,timeout,seed,k,epsilon,num_threads,imbalance,totalPartitionTime,objective,km1,cut,failed"
    
    def __init__(self, content: str, timelimit: int = None):
        self.data = {}
        for i, key in enumerate(self.HEADER.split(',')):
            value = content.split(',')[i].strip()
            # Try to convert to int or float if possible
            try:
                if '.' in value:
                    value = float(value)
                else:
                    value = int(value)
            except ValueError:
                pass
            self.data[key] = value
        self.data['timelimit'] = timelimit

    def get(self, param):
        return self.data[param]

def parse_results_file(file_path: str):
    results_array = []
    timelimit = file_path.split('.')[-2]
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            results_array += [BenchmarkRun(content=line, timelimit=int(timelimit))]
    return results_array

def aggregate_runs(directory_path: str):
    runs = {}
    for file_path in glob.glob(directory_path + "/*.results"):
        file_name = file_path.split('/')[-1]
        instance_name = '.'.join(file_name.split('.')[:-2])
        if instance_name not in runs:
            runs[instance_name] = {"short": None, "long": None}

        run_results = parse_results_file(file_path)
        if len(run_results) > 1:
            min_km1=min([run.get('km1') for run in run_results if run.get('failed') == 'no'])
            runs[instance_name]["short"] = min_km1
        elif run_results:
            runs[instance_name]["long"] = run_results[0].get('km1')
        else:
            pass
            #print(f"Warning: No results in file {file_path}")

    return runs

### Various different results analysis ###

In [ ]:
RESULT_FILES_DIR = f"{HOME_DIR}/Documents/2025-10-17_test_results/mt_kahypar_evo_results"

# Aggregate runs from result files
runs = aggregate_runs(RESULT_FILES_DIR)

diff_array = []
for instance_name, results in runs.items():
    short_km1 = results["short"]
    long_km1 = results["long"]
    if short_km1 is not None and long_km1 is not None:
        diff = long_km1 - short_km1
        diff_array.append(diff)

# How often is long run better than short runs (percentage)
if diff_array:
    better_count = sum(1 for d in diff_array if d < 0)
    percentage_better = (better_count / len(diff_array)) * 100
    print(f"Percentage of instances where long run is better than short runs: {percentage_better:.2f}%")
else:
    print("No valid differences to analyze.")

print ("diff_array:", diff_array)

# Analyze largest relative differences (in favor of long runs)
relative_diffs = []
for run, km1 in runs.items():
    short_km1 = km1["short"]
    long_km1 = km1["long"]
    if short_km1 is not None and long_km1 is not None:
        diff = long_km1 - short_km1
        relative_diff = diff / short_km1
        relative_diffs.append((run, relative_diff))
relative_diffs.sort(key=lambda x: x[1], reverse=True)
print("relative differences: ", relative_diffs[:10])



Percentage of instances where long run is better than short run: 87.18%
diff_array: [-885, 84, -1568, -1115, -596, -5, 0, -137, -494, -286, -4, -51, 7, -3, -235, -440, -297, -162, -964, -1, -154, -20, -92, 0, -203, -918, -33, -2, -211, 15, -2117, -414, -619, -576, -93, -296, -344, -369, -35, -289, -752, 48, -4535, -126, -256, -152, -53, -517, -14, -516, -148, -31, -291, 5, -141, -764, 145, -561, -13, -1007, -116, -13, -1317, -583, -29, -610, -541, -2, -144, -58, 162, -277, -457, -87, -2, 87, -36, -1184]
relative differences:  [('c-61.mtx.hgr.1.8.1', 0.02957373036916174), ('sat14_6s153.cnf.primal.hgr.1.8.1', 0.0210896309314587), ('gearbox.mtx.hgr.1.8.1', 0.015112037519541427), ('sme3Db.mtx.hgr.1.8.1', 0.012618296529968454), ('opt1.mtx.hgr.1.32.1', 0.010999456816947311), ('sat14_6s133.cnf.dual.hgr.1.8.1', 0.005335365853658537), ('sat14_atco_enc2_opt1_05_21.cnf.primal.hgr.1.8.1', 0.0040727667662231876), ('lhr14.mtx.hgr.1.32.1', 0.002086811352253756), ('hvdc1.mtx.hgr.1.8.1', 0.0), ('opt1.m